In [1]:
import pandas as pd
import numpy as np
import json
import pickle
from pandas.io.json import json_normalize
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
import sys
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot as plt
!{sys.executable} -m pip install xgboost
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot as plt

In [2]:
with open("df_application_v2.pkl", 'rb') as picklefile: 
    df_application_v2 = pickle.load(picklefile)

In [4]:
with open("installments_payments.pkl", 'rb') as picklefile: 
    installments_payments_df = pickle.load(picklefile)

In [6]:
installments_payments_df.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [47]:
installments_payments_df["DAYS_INSTALMENT"] = installments_payments_df["DAYS_INSTALMENT"] * -1

In [49]:
installments_payments_df["DAYS_ENTRY_PAYMENT"] = installments_payments_df["DAYS_ENTRY_PAYMENT"] * -1

In [50]:
installments_payments_df["NUM_INSTALMENT_VERSION"].unique()

array([  1.,   0.,   2.,   4.,   3.,   5.,   7.,   8.,   6.,  13.,   9.,
        21.,  22.,  12.,  17.,  18.,  11.,  14.,  34.,  33.,  19.,  16.,
        15.,  10.,  26.,  27.,  20.,  25.,  23.,  24.,  31.,  32.,  28.,
        35.,  29.,  30.,  43.,  39.,  40.,  36.,  41.,  42.,  37.,  38.,
        68.,  44.,  45.,  46., 178.,  52.,  51.,  53.,  54.,  49.,  50.,
        58.,  57.,  55.,  56.,  48.,  47.,  72.,  59.,  73.,  61.])

In [51]:
installments_payments_df.loc[installments_payments_df["SK_ID_PREV"] == 1330831].head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,OVERDUE_PAYMENT_DAYS,UNDER_PAYMENT_TRACKER
1,1330831,151639,0.0,34,2156.0,2156.0,1716.525,1716.525,0.0,0.0
13877,1330831,151639,0.0,47,1761.0,1761.0,1716.525,1716.525,0.0,0.0
40913,1330831,151639,0.0,6,2827.0,2835.0,3375.000,3375.000,-8.0,0.0
41909,1330831,151639,0.0,76,877.0,877.0,1716.525,1716.525,0.0,0.0
98422,1330831,151639,0.0,72,999.0,999.0,1716.525,1716.525,0.0,0.0


In [52]:
#Here, the larger the negative number, the better. indicates the payment was made well before the due date. 
installments_payments_df["OVERDUE_PAYMENT_DAYS"] = installments_payments_df["DAYS_ENTRY_PAYMENT"] - installments_payments_df["DAYS_INSTALMENT"]


In [53]:
installments_payments_df["UNDER_PAYMENT_TRACKER"] = installments_payments_df["AMT_INSTALMENT"] - installments_payments_df["AMT_PAYMENT"]

In [54]:
installments_payments_df.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,OVERDUE_PAYMENT_DAYS,UNDER_PAYMENT_TRACKER
0,1054186,161674,1.0,6,1180.0,1187.0,6948.360,6948.360,7.0,0.000
1,1330831,151639,0.0,34,2156.0,2156.0,1716.525,1716.525,0.0,0.000
2,2085231,193053,2.0,1,63.0,63.0,25425.000,25425.000,0.0,0.000
3,2452527,199697,1.0,3,2418.0,2426.0,24350.130,24350.130,8.0,0.000
4,2714724,167756,1.0,2,1383.0,1366.0,2165.040,2160.585,-17.0,4.455


In [55]:
installments_payments_df.loc[installments_payments_df["SK_ID_PREV"] == 1381627]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,OVERDUE_PAYMENT_DAYS,UNDER_PAYMENT_TRACKER
1040325,1381627,187868,1.0,1,912.0,932.0,19393.470,19393.470,20.0,0.0
1833344,1381627,187868,2.0,2,882.0,897.0,86423.805,86423.805,15.0,0.0


In [56]:
temp_df = installments_payments_df[["SK_ID_PREV", "SK_ID_CURR"]]

In [57]:
temp_df.drop_duplicates(inplace = True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [58]:
temp_df.shape

(997752, 2)

In [59]:
grouped_installments_df = installments_payments_df.groupby(["SK_ID_PREV"])[["OVERDUE_PAYMENT_DAYS", "UNDER_PAYMENT_TRACKER"]].mean().reset_index()

In [60]:
grouped_installments_df = pd.merge(grouped_installments_df, temp_df, how="left", on=["SK_ID_PREV"])

In [61]:
grouped_installments_df.head()

,SK_ID_PREV,OVERDUE_PAYMENT_DAYS,UNDER_PAYMENT_TRACKER,SK_ID_CURR
0,1000001,16.000000,0.000000,158271
1,1000002,19.750000,0.000000,101962
2,1000003,15.333333,0.000000,252457
3,1000004,26.714286,0.000000,260094
4,1000005,8.454545,1337.600455,176456


In [62]:
grouped_installments_df.loc[grouped_installments_df["SK_ID_CURR"] == 187868]

,SK_ID_PREV,OVERDUE_PAYMENT_DAYS,UNDER_PAYMENT_TRACKER,SK_ID_CURR
219052,1381627,17.500000,0.000000,187868
284790,1491169,24.000000,0.000000,187868
289749,1499074,25.000000,-2378.853000,187868
358081,1625177,1.000000,-13106.250000,187868
373593,1660097,18.857143,-3535.714286,187868
739580,2349620,13.750000,-11298.877500,187868
784956,2437218,26.000000,0.000000,187868
838559,2539207,8.000000,-16968.056538,187868
908241,2672704,13.250000,0.000000,187868
936553,2727121,6.000000,0.000000,187868


In [65]:
grouped_installments_df = grouped_installments_df.groupby(["SK_ID_CURR"])[["OVERDUE_PAYMENT_DAYS", "UNDER_PAYMENT_TRACKER"]].mean().reset_index()

In [66]:
with open('grouped_installments.pkl', 'wb') as picklefile:
    pickle.dump(grouped_installments_df, picklefile)